In [1]:
import numpy as np
import copy
import math
import itertools
from functools import cache

In [2]:
class SequenceTreeNode:
    def __init__(self):
        self.children = np.array([None, None])
        self.char = -1
        self.height = 0
        self.size = 0
        self.contains_0_vector = False
        self.contains_1_vector = False

    def has_sequence(self, x):
        node = self
        for i,c in enumerate(x):
            if node.children[c] is None:
                return False
            node = node.children[c]
        return True

    def insert_sequence(self, x):
        if self.has_sequence(x):
            return

        self.__insert_sequence_internal(x)

    def __insert_sequence_internal(self, x):
        self.size += 1
        if x.size == 0:
            return

        self.height = x.size

        if np.all(x==0):
            self.contains_0_vector = True   
        elif np.all(x==1):
            self.contains_1_vector = True 

        letter = x[0]
        if self.children[letter] is None:
            self.children[letter] = SequenceTreeNode()

        self.children[letter].char = letter
        self.children[letter].__insert_sequence_internal(x[1:])

    def print_tree(self):
        self.__print_tree(0)

    def __print_tree(self, depth):
        print(f'({depth}) - char {self.char} has {self.size} sequences.\n\t\
            contains the 0 vector: {self.contains_0_vector}\n\t\
            contains the 1 vector: {self.contains_1_vector}')
        for child in self.children:
            if child is not None:
               child.__print_tree(depth+1)

    def print_all_sequences(self):
        self.__print_all_sequences(np.array([], dtype=np.ubyte))

    
    def __print_all_sequences(self, path):
        if np.all(self.children == None):
            print(path)
        for child in self.children:
            if child is None:
                continue
            child.__print_all_sequences(np.append(path, child.char))

    def decrease_size_by(self, k):
        if k <= 0:
            return

        self.size -= k

        if self.children[0] is not None and self.children[0].size <= k:
            self.children[1].decrease_size_by(k - self.children[0].size)
            self.children[0] = None
            return
        elif self.children[1] is not None and self.children[1].size <= k:
            self.children[0].decrease_size_by(k - self.children[1].size)
            self.children[1] = None
            return

        if self.children[0] is not None:
            self.children[0].decrease_size_by(k)
        else: 
            self.children[1].decrease_size_by(k)
        

    def decrease_size_to(self, N):
        self.decrease_size_by(self.size - N)

    def get_unique_path(self):
        if self.size != 1:
            raise Exception()

        path = []

        node = self
        while node.height > 0:
            if node.children[0] is not None:
                path.append(0)
                node = node.children[0]
            else:
                path.append(1)
                node = node.children[1]
        
        return np.array(path, dtype=int)

In [3]:
U = SequenceTreeNode()
U.insert_sequence(np.array([0, 1, 0, 1, 1, 0]))
print(U.get_unique_path())

[0 1 0 1 1 0]


In [4]:
@cache
def get_maximal_deletion_ball_size(n, t):
    if n < t or t < 0:
        return 0

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i)
    return size

@cache
def get_maximal_number_of_common_subsequences(n, t):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t) - get_maximal_deletion_ball_size(n-1,t) + get_maximal_deletion_ball_size(n-2,t-1)

In [5]:
def get_ordering(U):
    k = np.zeros_like(U.children)
    for i, child in enumerate(U.children):
        if child is None:
            k[i] = 0
        else:
            k[i] = child.size
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

In [6]:
def get_deletion_ball_tree(x, t, N=-1):
    result = SequenceTreeNode()
    for c in itertools.combinations(x, x.size-t):
        result.insert_sequence(np.array(c))
        if result.size == N:
            break

    return result

In [7]:
n = 10
t = 2
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)
D_x = get_deletion_ball_tree(x, t)
print(f'x = {x}')
print()
D_x.print_all_sequences()

x = [1 0 0 0 1 0 1 1 0 1]

[0 0 0 0 1 1 0 1]
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 1 0]
[0 0 0 1 0 1 1 1]
[0 0 0 1 1 1 0 1]
[0 0 1 0 1 1 0 1]
[1 0 0 0 0 1 0 1]
[1 0 0 0 0 1 1 0]
[1 0 0 0 0 1 1 1]
[1 0 0 0 1 0 0 1]
[1 0 0 0 1 0 1 0]
[1 0 0 0 1 0 1 1]
[1 0 0 0 1 1 0 1]
[1 0 0 0 1 1 1 0]
[1 0 0 0 1 1 1 1]
[1 0 0 1 0 1 0 1]
[1 0 0 1 0 1 1 0]
[1 0 0 1 0 1 1 1]
[1 0 0 1 1 1 0 1]
[1 0 1 0 1 1 0 1]


In [8]:
def get_u_a_i(U, a, i):
    root = SequenceTreeNode()
    node = U
    for j in range(i-1):
        node = node.children[1-a]        
        if node is None:
            return root
    
    node = node.children[a]
    if node is None:
        return root

    root = copy.copy(node)
    root.char = -1
    
    return root

In [9]:
# n = 10
# t = 1
# x=np.random.randint(0,q,(n),dtype=np.ubyte)
# D_x = get_deletion_ball_tree(x, t)
# print(D_x.size)
# D_x.print_all_sequences()
# print()
# D_x.decrease_size_by(2)
# print(D_x.size)
# D_x.print_all_sequences()
# print()
# u_a_i = get_u_a_i(D_x, 0, 1)
# u_a_i.print_all_sequences()
# u_a_i.decrease_size_to(u_a_i.size)
# print()
# u_a_i.print_all_sequences()


In [10]:
def get_subsequence_reconstruction_threshold(n, t, order_comp):
    for i, w_i in enumerate(order_comp):
        tau_i = get_maximal_number_of_common_subsequences(n-i-1,t-i)
        if w_i > tau_i:
            return i

In [11]:
def reconstruct_x_from_subsequences(n, U, verbose=False):
    t = n-U.height
    reconstruction = np.array([], dtype=int)
    N = get_maximal_number_of_common_subsequences(n,t)+1
    while t >= 1:
        order_perm, order_comp = get_ordering(U)
        if verbose:
            print(f'n: {n}, t: {t}, U: ({U.size}, {U.height}), N: {N}')
            print(f'perm: {order_perm}, comp: {order_comp}')
        j = get_subsequence_reconstruction_threshold(n, t, order_comp)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if verbose:
            print(j, reconstruction)

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t)+1
        U = get_u_a_i(U, order_perm[j], 1)
        U.decrease_size_to(N)
        
    if verbose:
        print(U.size, U.get_unique_path())

    return np.concatenate((reconstruction, U.get_unique_path()))

In [12]:
n = 100
t = 3
q = 2

x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t)+1
print(N)
D_x = get_deletion_ball_tree(x, t, N)

while D_x.size < N:
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball_tree(x, t, N)

print(D_x.size)

9315
9315


In [13]:
# print(subsequences.shape)
# print(get_next_subsequnces(subsequences,0).shape)
# print(get_next_subsequnces(subsequences,0))
D_x = get_deletion_ball_tree(x, t, N)
print(x)
print(n, D_x.size)
reconstructed_x = reconstruct_x_from_subsequences(n, D_x, False)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

[0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0
 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1]
100 9315
Reconstructed [0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0
 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1]
From [0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0
 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1]
True


In [14]:
def get_insertion_ball_tree(x, t, N=-1):
    result = SequenceTreeNode()
    n = x.shape[0]
    for indices in itertools.combinations_with_replacement(range(n+1), t):
        for letters in itertools.product(range(q), repeat=t):
            seq = np.insert(x, indices, letters)
            result.insert_sequence(seq)

            if result.size == N:
                return result

    return result

In [15]:
@cache
def get_maximal_insertion_ball_size(n, t):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)
    return size

In [16]:
@cache
def get_maximal_number_of_common_supersequences(n, t):
    if t <= 0:
        return 0
    result = 0
    for i in range(1, t+1):
        result += get_maximal_insertion_ball_size(n,t-i)
    return 2*result

In [17]:
n = 10
t = 1
x=np.random.randint(0,q,(n),dtype=np.ubyte)
I_x = get_insertion_ball_tree(x, t)
print(x)
print(I_x.size)
I_x.print_all_sequences()
print()

[0 1 1 0 1 1 0 1 1 0]
12
[0 0 1 1 0 1 1 0 1 1 0]
[0 1 0 1 0 1 1 0 1 1 0]
[0 1 1 0 0 1 1 0 1 1 0]
[0 1 1 0 1 0 1 0 1 1 0]
[0 1 1 0 1 1 0 0 1 1 0]
[0 1 1 0 1 1 0 1 0 1 0]
[0 1 1 0 1 1 0 1 1 0 0]
[0 1 1 0 1 1 0 1 1 0 1]
[0 1 1 0 1 1 0 1 1 1 0]
[0 1 1 0 1 1 1 0 1 1 0]
[0 1 1 1 0 1 1 0 1 1 0]
[1 0 1 1 0 1 1 0 1 1 0]



In [18]:
def get_m_vector(U, t, a):
    m_vec = np.zeros((t+1,), dtype=np.uint)
    for i in range(t+1):
        u_a_i = get_u_a_i(U, a, i+1)
        m_vec[i] = u_a_i.size

    return m_vec

In [19]:
def get_first_x(U, t):
    N = U.size
    candidates = []
    for i in range(2):
        m_v = get_m_vector(U, t, i)
        if m_v.sum() == N:
            candidates.append((i, m_v))

    if len(candidates) == 1:
        return candidates[0]

    U_0_1 = 0
    U_1_0 = 0

    if U.children[0] is not None:
        U_0_1 = U.children[0].size - (1 if U.contains_0_vector else 0)
    if U.children[1] is not None:
        U_1_0 = U.children[1].size - (1 if U.contains_1_vector else 0)

    # Since len(candidates)>1 then both 0 and 1 are candidates 
    # and because we added them by order candidates[0] represents 0
    if U_0_1 > U_1_0:
        return candidates[0] 
    
    return candidates[1]

In [20]:
def get_supersequence_resonstruction_threshold(n, t, m_v):
    t = m_v.size - 1
    for i, w_i in enumerate(m_v):
        tau_i = get_maximal_number_of_common_supersequences(n-1,t-i)
        if w_i > tau_i:
            return i

In [21]:
def reconstruct_x_from_supersequences(n, U, verbose=False):
    t = U.height-n
    N = get_maximal_number_of_common_supersequences(n,t)+1
    reconstruction = np.array([], dtype=int)
    while t >= 1:
        if verbose:
            print(f'n: {n}, t: {t}, U: ({U.size}, {U.height}), N: {N}')
        x_1, m_x_1 = get_first_x(U, t)
        reconstruction = np.append(reconstruction, x_1)
        if n == 1: 
            return reconstruction
        j = get_supersequence_resonstruction_threshold(n, t, m_x_1)
        if verbose:
            print(j, reconstruction)

        U = get_u_a_i(U, x_1, j+1)
        n = n-1
        t = t-j  
        N = get_maximal_number_of_common_supersequences(n,t)+1
        U.decrease_size_to(N)

    return np.concatenate((reconstruction, U.get_unique_path()))

In [22]:
n = 100
t = 3
q = 2

x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_supersequences(n,t)+1
a = get_maximal_insertion_ball_size(n,t)
print(N)
I_x = get_insertion_ball_tree(x, t, N)
print(a, I_x.size)
m_0 = get_m_vector(I_x, t, 0)
m_1 = get_m_vector(I_x, t, 1)
print(m_0, m_0.sum())
print(m_1, m_1.sum())

10715
182208 10715
[5461 5152  101    1] 10715
[5254  309  101  100] 5764


In [23]:
# print(subsequences.shape)
# print(get_next_subsequnces(subsequences,0).shape)
# print(get_next_subsequnces(subsequences,0))
I_x = get_insertion_ball_tree(x, t, N)
print(x)
print(n, I_x.size)
reconstructed_x = reconstruct_x_from_supersequences(n, I_x)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

[0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1
 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0]
100 10715
Reconstructed [0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1
 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0]
From [0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1
 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0]
True
